In [ ]:
%%capture 
!pip install --upgrade yt_dlp
!pip install pydub SpeechRecognition ffmpeg ffprobe googletrans==3.1.0a0 transformers
!pip install rouge
import yt_dlp
import time
import re
import os
from pydub import AudioSegment
import speech_recognition as sr
import math
from tqdm import tqdm
from googletrans import Translator
from threading import Thread
!pip install transformers[torch]==4.3
!pip install packaging==21.3
!pip install torch
!pip install sentencepiece
import torch
from transformers import T5Tokenizer
from transformers import T5ForConditionalGeneration
from transformers import T5Config
!pip install gtts
from gtts import gTTS
from IPython.display import Audio, display
from rouge import Rouge
import json

In [ ]:
!pip install anvil-uplink

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.1/90.1 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ws4py: filename=ws4py-0.5.1-py3-none-any.whl size=45228 sha256=092a0d38b1516fb92f2dbed42dd456fe86268d43bb660cf0d21ec8f29a622118
  Stored in directory: /root/.cache/pip/wheels/2e/7c/ad/d9c746276bf024d44296340869fcb169f1e5d80fb147351a57
Successfully built ws4py


In [ ]:
import anvil.server

anvil.server.connect("server_OGM6JCVQ6AH4Q4LREDA2GRAC-AQVTOKX2GAGASCF3")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default Environment" as SERVER


In [ ]:
url=''
@anvil.server.callable
def input_text(yurl): 
  url = yurl
  ydl_opts={}

  with yt_dlp.YoutubeDL(ydl_opts) as ydl:
          info_dict = ydl.extract_info(url, download=False)
  video_title = info_dict['title']
  video_name = re.sub('[\\\\/*?:"<>|]', '', video_title)
  name = video_name
  ydl_opts = {
     'format': 'm4a/bestaudio/best',
         'noplaylist': True,
         'continue_dl': True,
         'outtmpl': f'./{name}.wav',
         'postprocessors': [{
             'key': 'FFmpegExtractAudio',
             'preferredcodec': 'wav',
             'preferredquality': '192',
         }],
         'geobypass':True,
         'ffmpeg_location':'/usr/bin/ffmpeg'
  }

  with yt_dlp.YoutubeDL(ydl_opts) as ydl:
      error_code = ydl.download(url)
  #Splitting audio
  class SplitWavAudioMubin():
      def __init__(self, folder, filename):
          self.folder = folder
          self.filename = filename
          self.filepath = folder + filename
        # print(self.filepath)
          self.audio = AudioSegment.from_wav(self.filepath)
    
      def get_duration(self):
          return self.audio.duration_seconds
    
      def single_split(self, from_min, to_min, split_filename):
          t1 = from_min * 60 * 1000
          t2 = to_min * 60 * 1000
          split_audio = self.audio[t1:t2]
          split_audio.export(split_filename, format="wav")
        
      def multiple_split(self, min_per_split):
          total_mins = math.ceil(self.get_duration() / 60)
          for i in range(0, total_mins, min_per_split):
              split_fn = str(i) + '_' + self.filename
              self.single_split(i, i+min_per_split, split_fn)
              if i == total_mins - min_per_split:
                  print('All splited successfully')
          print('>>> Video duration: ' + str(self.get_duration()))
        
  def split_audio(file_name):
        directory = "splitted files for: " + file_name
        os.mkdir(directory)
        split_wav = SplitWavAudioMubin("/content/", file_name)
        split_wav.multiple_split(min_per_split=1)

  file_name = "{}.wav.wav".format(video_name)
  split_audio(file_name)
  search_dir = "./"
  files = filter(os.path.isfile, os.listdir(search_dir))
  files = [os.path.join(search_dir, f) for f in files] # add path to each file
  files.sort(key=lambda x: os.path.getmtime(x))

  def speech_recognizer(files, frames, i):  ## This function recognizes speech in our WAV files
    texts = []
    recognizer = sr.Recognizer()

    for file in tqdm(files):
        with sr.AudioFile(file) as source:
            recorded_audio = recognizer.listen(source)
        try:
            text = recognizer.recognize_google(
                recorded_audio,
                language="en-US"  ## Replace with language keyword
            )
            texts.append(text)
        except Exception as ex:
            print(ex)
    result = ""
    for text in texts:
        result += " " + text
    frames[i] = result
    return result

  def split_files(files, n_batches): ## This function split the files evenly between the threads we have.
    k, m = divmod(len(files), n_batches)
    return list(files[i*k+min(i, m):(i+1)*k+min(i+1, m)] for i in range(n_batches))

  def main(n_batches=8, verbose=True): ## By default, the maximum capacity of threads supported in collab is 8
    threads = [None]*n_batches
    frames = [None]*n_batches
    batches = split_files(files, n_batches)
    start = 0
    for i in range(len(batches)):
        if i>0:
            start_index=len(batches[i-1])
        else:
            start_index = 0
        t = Thread(target=speech_recognizer, args=(batches[i], frames, i))
        threads[i] = t
        t.start()
    for t in threads:
        t.join()
    return frames
  
  frames1 = main()
  result1 = ""
  for text in frames1:
      result1 += " " + text
  print(result1)
  os.chdir("../")
  text_file = open("Transcription_"+ file_name[:-4] +".txt", "w")
  text_file.write(result1)
  text_file.close()
  model = T5ForConditionalGeneration.from_pretrained('t5-large')
  tokenizer = T5Tokenizer.from_pretrained('t5-large')
  device = torch.device('cpu')
  text=result1

  preprocessed_text =text.strip().replace('\n','')
  t5_input_text = 'summarize: ' + preprocessed_text
  ml=len(t5_input_text.split())
  tokenized_text = tokenizer.encode(t5_input_text, return_tensors='pt', max_length=ml).to(device)
  summary_ids = model.generate(tokenized_text, min_length=30, max_length=200)
  summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
  
  #return summary      
  #This module is imported so that we can
  #play the converted audio
  #Provide the string to convert to speech
  tts = gTTS(summary)                     
  #save the string converted to speech as a .wav file    
  tts.save('final.wav') 
  sound_file = 'final.wav'
  display(Audio(sound_file, autoplay=True))
  return summary


In [ ]:
@anvil.server.callable
def eval(fref,summary):
  reference=fref
  rouge = Rouge()
  r_score=rouge.get_scores(summary, reference)
  print(json.dumps(r_score))
  return json.dumps(r_score)

In [ ]:
anvil.server.wait_forever()

[youtube] Extracting URL: https://youtu.be/uwzViw-T0-A
[youtube] uwzViw-T0-A: Downloading webpage
[youtube] uwzViw-T0-A: Downloading android player API JSON
[youtube] Extracting URL: https://youtu.be/uwzViw-T0-A
[youtube] uwzViw-T0-A: Downloading webpage
[youtube] uwzViw-T0-A: Downloading android player API JSON
[info] uwzViw-T0-A: Downloading 1 format(s): 140
[dashsegments] Total fragments: 1
[download] Destination: ./Thirsty Crow Story in English  Moral stories for Kids  Bedtime Stories for Children.wav
[download] 100% of    2.24MiB in 00:00:00 at 15.62MiB/s              
[FixupM4a] Correcting container of "./Thirsty Crow Story in English  Moral stories for Kids  Bedtime Stories for Children.wav"
[ExtractAudio] Destination: ./Thirsty Crow Story in English  Moral stories for Kids  Bedtime Stories for Children.wav.wav
Deleting original file ./Thirsty Crow Story in English  Moral stories for Kids  Bedtime Stories for Children.wav (pass -k to keep)
All splited successfully
>>> Video dura

  0%|          | 0/1 [00:00<?, ?it/s]



  0%|          | 0/1 [00:00<?, ?it/s]





0it [00:00, ?it/s]


0it [00:00, ?it/s]




0it [00:00, ?it/s]
0it [00:00, ?it/s]



  0%|          | 0/1 [00:00<?, ?it/s]

0it [00:00, ?it/s]



100%|██████████| 1/1 [00:02<00:00,  2.21s/it]




100%|██████████| 1/1 [00:04<00:00,  4.08s/it]

100%|██████████| 1/1 [00:16<00:00, 16.11s/it]


  my kids the thirsty crystal story  my kids the thirsty crystal story  that the pro could not drink it then the Crow try to break the picture but he could not he tried again and again to overcome the picture but he was very little water at the bottom of the picture he looked around and saw some pebbles lying nearby he took the pebbles one by one and the picture the water which the neck of the  the crude brand water to his heart's contained and see you the moral of the stories where there is a will there is a way    


/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-large automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


[{"rouge-1": {"r": 0.375, "p": 0.25, "f": 0.29999999520000004}, "rouge-2": {"r": 0.12195121951219512, "p": 0.0641025641025641, "f": 0.08403360892874821}, "rouge-l": {"r": 0.34375, "p": 0.22916666666666666, "f": 0.27499999520000007}}]


KeyboardInterrupt: ignored